In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn import set_config
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
import matplotlib.pyplot as plt

In [ ]:
label_f = pd.read_csv('results/center_female.csv')
label_m = pd.read_csv('results/center_male.csv')
label_f['BA-CA'] = label_f['BA'] - label_f['CA']
label_m['BA-CA'] = label_m['BA'] - label_m['CA']

analysis

In [ ]:
from sksurv.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
import scipy.stats as st

In [ ]:
def hazaard_calculator(t2df, target, within= False):
   llabel = target+' '+'Label'
   # t2df = t2df[~t2df['VAT_label'].isin(['Pre-existing Disease','Post-DXA and Pre-Disease'])] # remove previous disease
   t2df = t2df[~t2df[llabel].isin(['Before'])] # remove previous disease
   # t2df= t2df[~t2df['VAT_label'].isin(['Normal', 'Hypernormal'])]
   if within:
      t2df = t2df[t2df[target+' '+'Duration']<= 365*within]
   print(t2df[llabel].value_counts())
   t2df['Status'] = t2df[llabel].apply(lambda x: True if x == 'After' else False)
   t2df['Survival_in_days'] = t2df[target+' '+'Duration']
   # # Create Model'
   # t2df['Sex_enbed'] = t2df['Sex'].apply(lambda x: 0 if x == 'Female' else 1)
   t2df = t2df[['CA',  'BA','BA-CA','VAT_Rate', 'lean_Rate','Diastolic blood pressure', 'BMI','Waist circumference',
       'Systolic blood pressure', 'Antihypertension','Hip circumference', 'Smoking','Status', 'Survival_in_days']]
   x = t2df.fillna(t2df.mean())
   # x = t2df.dropna()
   # print("Remove NA:", len(t2df)-len(x))
   x = OneHotEncoder().fit_transform(x)
   print(x['Status'].value_counts())
   iy = ['Status', 'Survival_in_days']
   kf = KFold(n_splits=5, random_state=42, shuffle=True)
   c_test_t = []
   c_test_v = []
   c_test_b = []
   for i, (train_index, test_index) in enumerate(kf.split(x)):
      print('Fold:', i)
      train = x.iloc[train_index]
      test = x.iloc[test_index]
      # print('CA')
      cph1 = CoxnetSurvivalAnalysis(l1_ratio=0.99, fit_baseline_model=True)
      int1 = ['CA', 'BMI', 'Diastolic blood pressure', 'Systolic blood pressure', 'Smoking', 'Waist circumference', 'Hip circumference']
      cph1.fit(train[int1], train[iy].to_records(index=False))
      # print(cph1)
      # print('BA')
      # int2 = ['CA', 'VAT_Rate', 'lean_Rate']
      int2 = ['VAT_Rate', 'lean_Rate', 'BMI', 'Diastolic blood pressure', 'Systolic blood pressure', 'Smoking', 'Waist circumference', 'Hip circumference']
      cph2 = CoxnetSurvivalAnalysis(l1_ratio=0.99, fit_baseline_model=True)
      cph2.fit(train[int2], train[iy].to_records(index=False))
      # int3 = ['BA', 'CA']
      int3 = ['BA', 'BMI', 'Diastolic blood pressure', 'Systolic blood pressure', 'Smoking', 'Waist circumference', 'Hip circumference']
   #    cph2 = CoxPHFitter()
      cph3 = CoxnetSurvivalAnalysis(l1_ratio=0.99, fit_baseline_model=True)
      cph3.fit(train[int3], train[iy].to_records(index=False))
      # print('Traditional C-Index')
      train_tradition = concordance_index_censored(train["Status"], train["Survival_in_days"], cph1.predict(train[int1]))
      # print(train_tradition)
      test_tradition = concordance_index_censored(test["Status"], test["Survival_in_days"], cph1.predict(test[int1]))
      train_vt = concordance_index_censored(train["Status"], train["Survival_in_days"], cph2.predict(train[int2]))
      test_vt = concordance_index_censored(test["Status"], test["Survival_in_days"], cph2.predict(test[int2]))
      train_ba = concordance_index_censored(train["Status"], train["Survival_in_days"], cph3.predict(train[int3]))
      test_ba = concordance_index_censored(test["Status"], test["Survival_in_days"], cph3.predict(test[int3]))
      # print(test_ba)
      c_test_t.append(test_tradition[0])
      c_test_v.append(test_vt[0])
      c_test_b.append(test_ba[0])
   print(np.mean(c_test_t) ,st.t.interval(alpha=0.95, df=len(c_test_t)-1, 
              loc=np.mean(c_test_t), 
              scale=st.sem(c_test_t)) )
   print(np.mean(c_test_v), st.t.interval(alpha=0.95, df=len(c_test_v)-1, 
              loc=np.mean(c_test_v), 
              scale=st.sem(c_test_v)))
   print(np.mean(c_test_b), st.t.interval(alpha=0.95, df=len(c_test_b)-1, 
              loc=np.mean(c_test_b), 
              scale=st.sem(c_test_b)))
   
   print(st.ttest_ind(c_test_t, c_test_b))
   print(st.ttest_ind(c_test_t, c_test_v))


In [ ]:
all_df = pd.concat([label_m, label_f],axis=0)


In [ ]:
def hazaard_calculator_all(t2df, target, within= False, n_splits=10):
   llabel = target+' '+'Label'
   t2df = t2df[~t2df[llabel].isin(['Before'])] # remove previous disease
   t2df= t2df[~t2df['VAT_label'].isin(['Normal', 'Hypernormal'])]
   if within:
      t2df = t2df[t2df[target+' '+'Duration']<= 365*within]
   print(t2df[llabel].value_counts())
   t2df['Status'] = t2df[llabel].apply(lambda x: True if x == 'After' else False)
   t2df['Survival_in_days'] = t2df[target+' '+'Duration']
   # # Create Model'
   t2df['Sex_enbed'] = t2df['Sex'].apply(lambda x: 0 if x == 'Female' else 1)
   t2df = t2df[['CA', 'Sex_enbed', 'BA','BA-CA','VAT_Rate', 'lean_Rate','Diastolic blood pressure', 'BMI','Waist circumference',
       'Systolic blood pressure', 'Antihypertension','Hip circumference', 'Smoking','Status', 'Survival_in_days']]
   x = t2df.fillna(t2df.mean())
   x = OneHotEncoder().fit_transform(x)
   print(x['Status'].value_counts())
   iy = ['Status', 'Survival_in_days']
   kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
   c_test_t = []
   c_test_v = []
   c_test_b = []
   for i, (train_index, test_index) in enumerate(kf.split(x)):
      print('Fold:', i)
      train = x.iloc[train_index]
      test = x.iloc[test_index]
      # print('CA')
      cph1 = CoxnetSurvivalAnalysis(l1_ratio=0.99, fit_baseline_model=True)
      int1 = ['CA','Sex_enbed', 'BMI', 'Diastolic blood pressure', 'Systolic blood pressure', 'Smoking', 'Waist circumference', 'Hip circumference']
      cph1.fit(train[int1], train[iy].to_records(index=False))
      # print(cph1)
      # print('BA')
      int2 = ['CA', 'VAT_Rate', 'lean_Rate', 'Sex_enbed']
      # int2 = ['VAT_Rate', 'lean_Rate','Sex_enbed', 'BMI', 'Diastolic blood pressure', 'Systolic blood pressure', 'Smoking', 'Waist circumference', 'Hip circumference']
      cph2 = CoxnetSurvivalAnalysis(l1_ratio=0.99, fit_baseline_model=True)
      cph2.fit(train[int2], train[iy].to_records(index=False))
      int3 = ['BA', 'BA-CA', 'CA', 'Sex_enbed']
      # int3 = ['BA', 'BMI','Sex_enbed', 'Diastolic blood pressure', 'Systolic blood pressure', 'Smoking', 'Waist circumference', 'Hip circumference']
   #    cph2 = CoxPHFitter()
      cph3 = CoxnetSurvivalAnalysis(l1_ratio=0.95, fit_baseline_model=True)
      cph3.fit(train[int3], train[iy].to_records(index=False))
      # print('Traditional C-Index')
      train_tradition = concordance_index_censored(train["Status"], train["Survival_in_days"], cph1.predict(train[int1]))
      # print(train_tradition)
      test_tradition = concordance_index_censored(test["Status"], test["Survival_in_days"], cph1.predict(test[int1]))
      train_vt = concordance_index_censored(train["Status"], train["Survival_in_days"], cph2.predict(train[int2]))
      test_vt = concordance_index_censored(test["Status"], test["Survival_in_days"], cph2.predict(test[int2]))
      train_ba = concordance_index_censored(train["Status"], train["Survival_in_days"], cph3.predict(train[int3]))
      test_ba = concordance_index_censored(test["Status"], test["Survival_in_days"], cph3.predict(test[int3]))
      # print(test_ba)
      c_test_t.append(test_tradition[0])
      c_test_v.append(test_vt[0])
      c_test_b.append(test_ba[0])
   print(np.mean(c_test_t) ,st.t.interval(alpha=0.95, df=len(c_test_t)-1, 
              loc=np.mean(c_test_t), 
              scale=st.sem(c_test_t)) )
   print(np.mean(c_test_v), st.t.interval(alpha=0.95, df=len(c_test_v)-1, 
              loc=np.mean(c_test_v), 
              scale=st.sem(c_test_v)))
   print(np.mean(c_test_b), st.t.interval(alpha=0.95, df=len(c_test_b)-1, 
              loc=np.mean(c_test_b), 
              scale=st.sem(c_test_b)))
   
   print(st.ttest_ind(c_test_t, c_test_b))
   print(st.ttest_ind(c_test_t, c_test_v))


In [ ]:
hazaard_calculator_all(all_df, 'ASCVD')
hazaard_calculator_all(all_df, 'MACE')


Male

In [ ]:
hazaard_calculator(label_m,'ASCVD')
hazaard_calculator(label_m,'MACE')


Female

In [ ]:
hazaard_calculator(label_f,'MACE')
hazaard_calculator(label_f,'ASCVD')
